In [1]:
chrome_driver = '/Users/hamsongmin/Codestates/DA/chromedriver'
URL = "https://play.google.com/store/apps/details?id=com.fatsecret.android"

In [2]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd
from tqdm import tqdm

In [3]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = driver.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = driver.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = driver.find_element_by_xpath('/html/body/div[1]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div[2]/div/span/span').click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
                
    except Exception as e:
        print("에러 발생: ", e)

In [4]:
# 크롬 드라이버 세팅
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(chrome_driver, options = chrome_options)
# 페이지 열기
driver.get(URL)
# 페이지 로딩 대기
wait = WebDriverWait(driver, 5)

# '리뷰 모두 보기' 버튼 렌더링 확인(path 수정 @2022-06-22)
all_review_button_xpath = '/html/body/c-wiz[2]/div/div/div[1]/div[2]/div/div[1]/c-wiz[4]/section/div/div/div[5]/div/div/button/span'
button_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_button_xpath)))
# '리뷰 모두 보기' 버튼 클릭
driver.find_element(By.XPATH,all_review_button_xpath).click()

# '리뷰 모두 보기' 페이지 렌더링 대기
all_review_page_xpath = '/html/body/div[4]/div[2]/div/div/div/div/div[2]'
page_loading_wait = wait.until(EC.element_to_be_clickable((By.XPATH, all_review_page_xpath)))

# 페이지 무한 스크롤 다운
modal = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='fysCi']")))
scroll(modal)

/var/folders/jf/1dqwy_5j7sq1d6hn9_7xdvq40000gn/T/ipykernel_61050/712538647.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver, options = chrome_options)


스크롤 완료


In [8]:
# html parsing하기
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

In [9]:
# html 데이터 저장
with open("/Users/hamsongmin/Codestates/DA/dataset/data_html_FS.html", "w", encoding = 'utf-8') as file:
    file.write(str(soup_source))

In [10]:
# 리뷰 데이터 클래스 접근
review_source = soup_source.find_all(class_ = 'RHo1pe')
# 리뷰 데이터 저장용 배열
dataset = []
# 데이터 넘버링을 위한 변수
review_num = 0 
# 리뷰 1개씩 접근해 정보 추출
for review in tqdm(review_source):
    review_num+=1
    # 리뷰 등록일 데이터 추출
    date_full = review.find_all(class_ = 'bp9Aid')[0].text
    date_year = date_full[0:4] # 연도 데이터 추출
    # 해당 단어가 등장한 인덱스 추출
    year_index = date_full.find('년')
    month_index = date_full.find('월')
    day_index = date_full.find('일')
    
    date_month = str(int(date_full[year_index+1:month_index])) # 월(Month) 데이터 추출
    # 월 정보가 1자리의 경우 앞에 0 붙이기(e.g., 1월 -> 01월)
    if len(date_month) == 1:
        date_month = '0' + date_month
    
    date_day = str(int(date_full[month_index+1:day_index])) # 일(Day) 데이터 추출 
    # 일 정보가 1자리의 경우 앞에 0 붙여줌(e.g., 7일 -> 07일)
    if len(date_day) == 1:
        date_day = '0' + date_day
    
    # 리뷰 등록일 full version은 최종적으로 yyyymmdd 형태로 저장
    date_full = date_year + date_month + date_day
    user_name = review.find_all(class_ = 'X5PpBb')[0].text # 닉네임 데이터 추출
    rating = review.find_all(class_ = "iXRFPc")[0]['aria-label'][10] # 평점 데이터 추출
    content = review.find_all(class_ = 'h3YV2d')[0].text # 리뷰 데이터 추출

    data = {
        "id": review_num, 
        "date": date_full,
        "dateYear": date_year,
        "dateMonth": date_month,
        "dateDay": date_day,
        "rating": rating,
        "userName": user_name,
        "content": content
    }
    dataset.append(data)

 87%|███████████████████████████████▏    | 2664/3071 [00:02<00:00, 965.34it/s]


IndexError: list index out of range

In [11]:
df = pd.DataFrame(dataset)
df.to_csv('/Users/hamsongmin/Codestates/DA/dataset/FatSecret_review_dataset.csv', encoding = 'utf-8-sig') # csv 파일로 저장

In [12]:
df = pd.read_csv('/Users/hamsongmin/Codestates/DA/dataset/FatSecret_review_dataset.csv', encoding = 'utf-8-sig')
df = df.drop(['Unnamed: 0'], axis = 1) # 불필요한 칼럼 삭제
df

,id,date,dateYear,dateMonth,dateDay,rating,userName,content
0,1,20220928,2022,9,28,5,Sein Lee,"몇년전에도 식이시작하려고했을때 가장많이 추천받았던 앱이었는데, 지금도 최고의 앱인듯..."
1,2,20220627,2022,6,27,5,sunny,입력이 편해서 좋아요 리포트는 보여지는 주기가 일주일 단위로 고정이라 누를 때마다 ...
2,3,20200812,2020,8,12,5,슈슈,음식물 바코드를 찍으면 알아서 계산해주고 없는 경우엔 본인이 추가해서 적을 수 있어...
3,4,20211120,2021,11,20,1,호학애사,"핸드폰 2대+태블릿 1대에 연동해서 쓰는데, 오류가 자주 뜸. 며칠 전 기록이 제일..."
4,5,20210504,2021,5,4,5,둉됴니,쓰기 편하고 칼로리를 식사대별로 나눌수 있으니 정말 좋습니다. 다만 기존에 있는 제...
...,...,...,...,...,...,...,...,...
2659,2660,20140117,2014,1,17,1,Google 사용자,안켜짐
2660,2661,20140121,2014,1,21,5,Google 사용자,좋다
2661,2662,20140407,2014,4,7,5,Google 사용자,좋음
2662,2663,20140106,2014,1,6,4,Google 사용자,괜찬음
